In [214]:
from random import randint
import torch
import numpy as np

Carbin_Count = 9  # 


#定义环境
class Environment:
    def __init__(self):
        self.state = None
        self.time_spend = 0  # 使用的总共时间 要惩罚时间的
        self.reset()

    def reset(self):
        # 每个船舱的重量
        self.time_spend = 0  # 使用的总共时间 要惩罚时间的
        self.state = torch.FloatTensor(np.random.uniform(1, 2, (1, Carbin_Count)))
        return self.state

    def is_over(self):
        # 判断state每一项是否小于0
        return torch.all(self.state < 0)

    def get_state_reword(self, action: torch.Tensor):
        # 当前状态的分数,比如为负数了还有人来卸载就应该扣分
        reword = 0
        for i in action:
            index = int(i)  # 映射到货轮
            if index == 0:
                continue
            # 动起来的奖励
            reword += 1
            if self.state[0][index - 1] < 0:
                reword += 1*(self.state[0][index - 1]*13)
        if self.is_over():
            # 惩罚时间 
            reword += 1000
        return reword

    def step(self, action):
        for i in action:
            index = int(i)  # 映射到货轮
            if index == 0:
                continue
            self.state[0][index - 1] -= 0.1
        self.time_spend += 1
        reword = self.get_state_reword(action)
        return self.state, reword, self.is_over(), None


class MyWrapper:
    N = 3  # 港机数量

    def __init__(self):
        self.env = Environment()
        self.step_n = 0

    def reset(self):
        state = self.env.reset()
        self.step_n = 0
        return state

    def step(self, action):
        # print(action)
        state, reward, terminated, info = self.env.step(action)
        over = terminated
        #限制最大步数
        self.step_n += 1
        if self.step_n > 200:
            over = True
        return state, reward, over

    def to_show_state(self):
        return '%s  %s  %s' % (str(self.env.state), str(self.env.time_spend), str(self.env.is_over()))


env = MyWrapper()

env.reset()
env.step(torch.full((1, 1), 0.5, dtype=torch.float32))
env.to_show_state()

'tensor([[1.9172, 1.7847, 1.2609, 1.9336, 1.4298, 1.9052, 1.9956, 1.1288, 1.0310]])  1  tensor(False)'

In [215]:
import torch


class A2C:

    def __init__(self, model_actor, model_critic, model_critic_delay,
                 optimizer_actor, optimizer_critic):
        self.model_actor = model_actor
        self.model_critic = model_critic
        self.model_critic_delay = model_critic_delay
        self.optimizer_actor = optimizer_actor
        self.optimizer_critic = optimizer_critic

        self.model_critic_delay.load_state_dict(self.model_critic.state_dict())
        self.requires_grad(self.model_critic_delay, False)

    def soft_update(self, _from, _to):
        for _from, _to in zip(_from.parameters(), _to.parameters()):
            value = _to.data * 0.99 + _from.data * 0.01
            _to.data.copy_(value)

    def requires_grad(self, model, value):
        for param in model.parameters():
            param.requires_grad_(value)

    def train_critic(self, state, reward, next_state, over):
        self.requires_grad(self.model_critic, True)
        self.requires_grad(self.model_actor, False)

        #计算values和targets
        value = self.model_critic(state)

        with torch.no_grad():
            target = self.model_critic_delay(next_state)
        target = target * 0.99 * (1 - over) + reward
        # print('xxxx', value.size(), target.size(), reward.size())
        #时序差分误差,也就是tdloss
        loss = torch.nn.functional.mse_loss(value, target)

        loss.backward()
        self.optimizer_critic.step()
        self.optimizer_critic.zero_grad()
        self.soft_update(self.model_critic, self.model_critic_delay)

        #减去value相当于去基线
        return (target - value).detach()

    def train_actor(self, state, action, value):
        self.requires_grad(self.model_critic, False)
        self.requires_grad(self.model_actor, True)

        #重新计算动作的概率
        prob = self.model_actor(state)
        prob = prob.gather(dim=1, index=action)

        #根据策略梯度算法的导函数实现
        #函数中的Q(state,action),这里使用critic模型估算
        prob = (prob + 1e-8).log() * value
        loss = -prob.mean()

        loss.backward()
        self.optimizer_actor.step()
        self.optimizer_actor.zero_grad()

        return loss.item()


model_actor = [
    torch.nn.Sequential(
        torch.nn.Linear(9, 6 * 64),
        torch.nn.ReLU(),
        torch.nn.Linear(6 * 64, 6 * 64),
        torch.nn.ReLU(),
        torch.nn.Linear(6 * 64, Carbin_Count + 1),
        torch.nn.Softmax(dim=1),
    ) for _ in range(env.N)
]

model_critic, model_critic_delay = [
    torch.nn.Sequential(
        torch.nn.Linear(9, 6 * 64),
        torch.nn.ReLU(),
        torch.nn.Linear(6 * 64, 6 * 64),
        torch.nn.ReLU(),
        torch.nn.Linear(6 * 64, 1),
    ) for _ in range(2)
]

optimizer_actor = [
    torch.optim.Adam(model_actor[i].parameters(), lr=1e-3)
    for i in range(env.N)
]
optimizer_critic = torch.optim.Adam(model_critic.parameters(), lr=5e-3)

a2c = [
    A2C(model_actor[i], model_critic, model_critic_delay, optimizer_actor[i],
        optimizer_critic) for i in range(env.N)
]
# x = torch.FloatTensor([1,2,3,4,6,7,8,8,4]).resize(1,Carbin_Count)
# print(model_actor[0](x))
model_actor = None
model_critic = None
model_critic_delay = None
optimizer_actor = None
optimizer_critic = None

a2c


In [216]:

import random


#玩一局游戏并记录数据
def play(show=False):
    state = []
    action = []
    reward = []
    next_state = []
    over = []

    s = env.reset()
    o = False
    while not o:
        a = []
        for i in range(env.N):
            #计算动作
            prob = a2c[i].model_actor(torch.FloatTensor(s).reshape(
                1, -1))[0].tolist()
            # print(s, prob)
            a.append(random.choices(range(Carbin_Count + 1), weights=prob, k=1)[0])

        #执行动作
        ns, r, o = env.step(a)

        state.append(s)
        action.append(a)
        reward.append(r)
        next_state.append(ns)
        over.append(o)

        s = ns

        if show:
            print(env.to_show_state())
    # print(state[0])
    # print(type(state), len(state))
    state = torch.tensor([item.numpy() for item in state])
    action = torch.LongTensor(action).unsqueeze(-1)
    reward = torch.FloatTensor(reward).unsqueeze(-1).unsqueeze(-1)
    next_state = torch.tensor([item.numpy() for item in next_state])
    over = torch.LongTensor(over).reshape(-1, 1)

    return state, action, reward, next_state, over, reward.sum().item()


state, action, reward, next_state, over, reward_sum = play()

reward_sum, state.size(), reward.size(), action.size()

(690.7562866210938,
 torch.Size([77, 1, 9]),
 torch.Size([77, 1, 1]),
 torch.Size([77, 3, 1]))

In [217]:
def train():
    #训练N局
    for epoch in range(3_000):
        state, action, reward, next_state, over, _ = play()

        #合并部分字段
        state_c = state.flatten(start_dim=1)
        reward_c = reward.sum(dim=1)
        next_state_c = next_state.flatten(start_dim=1)

        for i in range(env.N):
            value = a2c[i].train_critic(state_c, reward_c, next_state_c, over)
            loss = a2c[i].train_actor(state_c, action[:, i], value)

        if epoch % 250 == 0:
            test_result = sum([play()[-1] for _ in range(20)]) / 20
            print(epoch, loss, test_result)


train()

0 22.58664321899414 496.131237411499
250 -0.0991472527384758 818.4802947998047
500 -1.3202204704284668 527.8281219482421
750 -1.4276803731918335 634.3708930969239
1000 -5.746939182281494 556.3465599060058
1250 5.100186824798584 677.1164047241211
1500 -3.825230836868286 667.8517028808594
1750 -11.097969055175781 627.5514198303223
2000 -17.649765014648438 648.3254898071289
2250 5.3272013664245605 656.0742874145508
2500 5.119644641876221 816.2438751220703
2750 -2.265223264694214 793.3619720458985


In [218]:
play(True)[-1]

tensor([[1.5261, 1.2775, 1.4488, 1.5083, 1.4671, 1.6141, 1.2891, 1.7750, 1.0300]])  1  tensor(False)
tensor([[1.4261, 1.2775, 1.4488, 1.5083, 1.3671, 1.6141, 1.2891, 1.6750, 1.0300]])  2  tensor(False)
tensor([[1.3261, 1.2775, 1.4488, 1.5083, 1.2671, 1.6141, 1.2891, 1.5750, 1.0300]])  3  tensor(False)
tensor([[1.3261, 1.2775, 1.4488, 1.5083, 1.2671, 1.5141, 1.2891, 1.4750, 0.9300]])  4  tensor(False)
tensor([[1.3261, 1.2775, 1.3488, 1.5083, 1.2671, 1.5141, 1.2891, 1.3750, 0.8300]])  5  tensor(False)
tensor([[1.2261, 1.2775, 1.3488, 1.5083, 1.2671, 1.5141, 1.2891, 1.2750, 0.7300]])  6  tensor(False)
tensor([[1.2261, 1.2775, 1.3488, 1.5083, 1.1671, 1.4141, 1.2891, 1.1750, 0.7300]])  7  tensor(False)
tensor([[1.1261, 1.2775, 1.3488, 1.4083, 1.0671, 1.4141, 1.2891, 1.1750, 0.7300]])  8  tensor(False)
tensor([[1.1261, 1.2775, 1.3488, 1.4083, 0.9671, 1.3141, 1.2891, 1.0750, 0.7300]])  9  tensor(False)
tensor([[1.1261, 1.2775, 1.3488, 1.4083, 0.9671, 1.2141, 1.2891, 0.9750, 0.6300]])  10  ten

692.7479248046875